<a href="https://colab.research.google.com/github/mazy1998/Solving-Optimization-Problems-with-JAX/blob/master/JaxOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd, jacrev
from jax.numpy import linalg

from numpy import nanargmin,nanargmax 

key = random.PRNGKey(42)

# Single Variable Optimization

Defining the Object Functions

In [0]:
def y(x):
  return ((x * np.sqrt(12*x - 36 )) / (2*(x - 3)))
def L(x):
  return np.sqrt( x**2 + y(x)**2)

### Solving with Gradient Descent
Using ***grad*** to find the derivative of the function ***L***

Using ***vmap*** to map the ***minGD*** function over the ***domain***

Using the gradient descent equation:

$x_{n+1} = x_{n} - 0.01 L^{'}(x_{n})$

In [0]:
gradL = grad(L)

def minGD(x): return x - 0.01 * gradL(x)

domain = np.linspace(3.0, 5.0, num=50)

vfuncGD = vmap(minGD)
#Recurrent loop of gradient descent
for epoch in range(500):
  domain = vfuncGD(domain)

minfunc = vmap(L)
minimums = minfunc(domain)

Finding the argmin and the objective minimum

In [0]:
arglist = nanargmin(minimums)
argmin = domain[arglist]
minimum = minimums[arglist]

print("The minimum is {} the argmin is {}".format(minimum,argmin))

The minimum is 7.794247150421143 the argmin is 4.505752086639404


### Solving with Newton's Method
Using ***grad*** to find the derivative of the function ***L***

Using ***vmap*** to map the ***minGD*** function over the ***domain***

Using the gradient descent equation:

$x_{n+1} = x_{n} - \frac{L^{'}(x_{n})}{L^{''}(x_{n})} $

In [0]:
gradL = grad(L)
gradL2 = grad(gradL)

def minNewton(x): return x - gradL(x)/gradL2(x)

domain = np.linspace(3.0, 5.0, num=50)
vfuncNT = vmap(minNewton)
for epoch in range(50):
  domain = vfuncNT(domain)

minimums = minfunc(domain)

Finding the argmin and the objective minimum

In [0]:
arglist = nanargmin(minimums)
argmin = domain[arglist]
minimum = minimums[arglist]

print("The minimum is {} the argmin is {}".format(minimum,argmin))

The minimum is 7.794229030609131 the arg min is 4.5


# Multivariable Optimization

Defining the Object Function

In [0]:
def paraboloid(x): return (x[0]*x[1]-2)**2 + (x[1]-3)**2
minfunc = vmap(paraboloid)

J = jacfwd(paraboloid)

### Solving with Gradient Descent using the Jacobian ($\nabla f$)
Using ***grad*** to find the jacobian of the function ***paraboloid***

Using ***vmap*** to map the ***minJacobian*** function over the ***domain***

Using the gradient descent equation:

$X_{n+1} = X_{n} - 0.01\nabla f(X_{n}) $

Where $ X = \left[x_1,x_2,\ldots,x_n \right]^T$

In [0]:
def minJacobian(x): return x - 0.1*J(x)  

domain = random.uniform(key, shape=(50,2), dtype='float32',
                        minval=-5.0, maxval=5.0)

vfuncHS = vmap(minJacobian)
for epoch in range(150):
  domain = vfuncHS(domain)


minimums = minfunc(domain)

Finding the argmin and the objective minimum

In [0]:
arglist = nanargmin(minimums)
argmin = domain[arglist]
minimum = minimums[arglist]

print("The minimum is {} the arg min is ({},{})".format(minimum,argmin[0],argmin[1]))

The minimum is 0.0 the arg min is (0.6666666865348816,3.0)


Defining the Hessian as $\nabla (\nabla f) = \nabla^{2}f$

In [0]:
def hessian(f):
    return jacfwd(jacrev(f))
    
H = hessian(paraboloid)

### Solving with Newton's Method using the Hessian ($\nabla^{2} f$)
Using ***hessian*** to find the Hessian of the function ***paraboloid***

Using ***vmap*** to map the ***minHessian*** function over the ***domain***

Using the gradient descent equation:

$X_{n+1} = X_{n} - 0.1 H^{-1}(X_{n}) \nabla f(X_{n}) $

Where $ X = \left[x_1,x_2,\ldots,x_n \right]^T$

In [0]:
def minHessian(x): return x - 0.1*linalg.inv(H(x)) @ J(x)  


domain = random.uniform(key, shape=(50,2), dtype='float32',
                        minval=-5.0, maxval=5.0)

vfuncHS = vmap(minHessian)
for epoch in range(150):
  domain = vfuncHS(domain)


minimums = minfunc(domain)

Finding the argmin and the objective minimum

In [0]:
arglist = nanargmin(minimums)
argmin = domain[arglist]
minimum = minimums[arglist]

print("The minimum is {} the arg min is ({},{})".format(minimum,argmin[0],argmin[1]))

The minimum is 9.094947017729282e-13 the arg min is (0.6666664481163025,3.0000009536743164)


# Multivariable Constrained Optimization

Defining the Object Function $f(x)$
and The Constrained Function $g(x)$

The Lagrangian is defined as ***Lagrange*** $f(x) - \lambda g(x) = 0 $

Therefore using Newton's Method we solve for $Lagrange(x)=0$

Which is the same as minimizing the multivariable function $\nabla Lagrange(x)$

Thus the reccurent loop is:
$X_{n+1} = X_{n} - \nabla^{2} Lagrange^{-1}(X_{n}) \nabla Lagrange(X_{n}) $

Where $ X = \left[x_1,x_2,\ldots,x_n \right]^T$


In [0]:
def f(x): return 4*(x[0]**2)*x[1]
def g(x): return x[0]**2 + x[1]**2 - 3

minfunc = vmap(f)

def Lagrange(l): return f(l[0:2]) - l[3]*g(l[0:2])

L = jacfwd(Lagrange)
gradL = jacfwd(L)

In [0]:
def solveLagrangian(l): return l - linalg.inv(gradL(l)) @ L(l)  


domain = random.uniform(key, shape=(50,3), dtype='float32',
                        minval=-5.0, maxval=5.0)

vfuncsLAG = vmap(solveLagrangian)
for epoch in range(150):
  domain = vfuncsLAG(domain)

minimums = minfunc(domain)


Finding the argmin and the objective minimum

In [0]:
arglist = nanargmin(minimums)
argmin = domain[arglist]
minimum = minimums[arglist]

print("The minimum is {}, the arg min is ({},{}), the lagrangian is {}".format(minimum,argmin[0],argmin[1],argmin[2]))

The minimum is -7.999999523162842, the arg min is (-1.4142135381698608,-1.0), the lagrangian is -4.0


# Solving a Three Variable Multivariable Constrained Optimization

Find the dimensions of the box with largest volume if the total surface area is $64 cm^2$

$Volume = f(x_0,x_1,x_2) = x_0 x_1x_2$

$Surface Area = g(x_0,x_1,x_2) = 2x_0x_1 + 2x_1x_1 + 2x_0x_2 = 64$

In [0]:
def f(x): return x[0]*x[1]*x[2]
def g(x): return 2*x[0]*x[1] + 2*x[1]*x[2] + 2*x[0]*x[2] - 64

minfunc = vmap(f)

def Lagrange(l): return f(l[0:3]) - l[3]*g(l[0:3])

L = jacfwd(Lagrange)
gradL = jacfwd(L)

In [0]:
def solveLagrangian(l): return l - 0.1*linalg.inv(gradL(l)) @ L(l)  

domain = random.uniform(key, shape=(50,4), dtype='float32',
                        minval=0, maxval=10)

vfuncsLAG = vmap(solveLagrangian)
for epoch in range(200):
  domain = vfuncsLAG(domain)


maximums = minfunc(domain)

In [0]:
arglist = nanargmax(maximums)
argmin = domain[arglist]
minimum = maximums[arglist]

print("The minimum is {}, the argmin is ({},{},{}), the lagrangian is {}".format(minimum,argmin[0],
                                                                                         argmin[1],
                                                                                         argmin[2],
                                                                                         argmin[3]))

The minimum is 34.83720016479492, the argmin is (3.2659873962402344,3.2659854888916016,3.2659873962402344), the lagrangian is 0.8164968490600586


It should be noted that this gives a 0.0000118855014% error!

# Multivariable MultiConstrained Optimization

Let's start by trying to maximize the object function $f(x_0,x_1)$ with the constraints $g(x_0,x_1)$ and $h(x_0,x_1)$. 

$f(x_0,x_1) = 13x_0*2 + 10x_0x_1+ 7x_1^2 + x_0 + x_1 +2$

$g(x_0,x_1) = 2x_0 - 5x_1 - 2 $

$h(x_0,x_1) = x_0 + x_1 -1$


In [0]:
def f(x) : return 13*x[0]**2 + 10*x[0]*x[1] + 7*x[1]**2 + x[0] + x[1]
def g(x) : return 2*x[0]-5*x[1]-2
def h(x) : return x[0] + x[1] -1

minfunc = vmap(f)

def Lagrange(l): return f(l[0:2]) - l[2]*g(l[0:2]) - l[3]*h(l[0:2])

L = jacfwd(Lagrange)
gradL = jacfwd(L)

In [0]:
def solveLagrangian(l): return l - 0.1*linalg.inv(gradL(l)) @ L(l)  


domain = random.uniform(key, shape=(300,4), dtype='float32',
                        minval=-4, maxval=1)


vfuncsLAG = vmap(solveLagrangian)
for epoch in range(300):
  domain = vfuncsLAG(domain)


maximums = minfunc(domain)

In [98]:
arglist = nanargmin(maximums)
argmin = domain[arglist]
minimum = maximums[arglist]

print("The minimum is {}, the argmin is ({},{}), the lagrangians are {} and {}".format(minimum,argmin[0],
                                                                                         argmin[1],
                                                                                         argmin[2],
                                                                                         argmin[3]))

The minimum is 13.999992370605469, the argmin is (0.9999997019767761,-1.18244605218365e-08), the lagrangians are 2.2857134342193604 and 22.428564071655273
